# 🧠 Transfer Learning with PyTorch

Welcome to this beginner-friendly notebook where we'll explore **Transfer Learning** using **PyTorch**. 

### 👇 What you'll learn:
- What is transfer learning?
- How to use a pretrained model (ResNet18)
- How to replace the classifier head
- How to train it on a small dataset (e.g., CIFAR-10 subset)

## 🔍 Step 1: Import Libraries

In [ ]:
import torch 
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

from torchvision import models


import numpy as np 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


cpu


# Transform - Preprocessing 

In [58]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.4],std = [0.229, 0.224, 0.225])
])

## 📥 Step 2: Load Dataset (CIFAR-10 Subset)

In [59]:
trainset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download= True,
    transform= transform
)

subset_indices = torch.randperm(len(trainset))[:2000]  # use smaller set
trainloader = torch.utils.data.DataLoader(
    torch.utils.data.Subset(trainset, subset_indices),
    batch_size=32, shuffle=True)

100%|██████████| 170498071/170498071 [00:07<00:00, 22112309.69it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data


## 🧠 Step 3: Load Pretrained ResNet18 and Modify Classifier

In [60]:
model = models.resnet18()
print(model.parameters)

<bound method Module.parameters of ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)


##  Function to Calculate the parameters

In [61]:
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params



## Info and Parameters

In [62]:
print(" Final Layer: ", model.fc)

print("Number of classes: ", model.fc.out_features)

total_params, trainable_params = count_parameters(model)

print("Total Parms: ", total_params)
print("Trainable Parms: ", trainable_params)

 Final Layer:  Linear(in_features=512, out_features=1000, bias=True)
Number of classes:  1000
Total Parms:  11689512
Trainable Parms:  11689512


# Freezing layers

In [64]:
for parm in model.parameters():
    parm.requires_grad = False

## 🧠 Step 3:  Modify Classifier

In [65]:
total_params, trainable_params = count_parameters(model)

print("Total Parms: ", total_params)
print("Trainable Parms: ", trainable_params)

Total Parms:  11689512
Trainable Parms:  0


In [66]:
model_tl = model
num_ftrs = model_tl.fc.in_features

print("Num of features:", num_ftrs)

model_tl.fc = nn.Linear(num_ftrs,10)

Num of features: 512


In [67]:
print(" Final Layer: ", model.fc)

print("Number of classes: ", model.fc.out_features)

total_params, trainable_params = count_parameters(model)

print("Total Parms: ", total_params)
print("Trainable Parms: ", trainable_params)

 Final Layer:  Linear(in_features=512, out_features=10, bias=True)
Number of classes:  10
Total Parms:  11181642
Trainable Parms:  5130


In [74]:
model_tl = model_tl.to(device=device)

## 🏋️‍♂️ Step 4: Train the New Classifier

In [ ]:
crit = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model_tl.fc.parameters(), lr=0.001)
epochs = 100
for epoch in range(epochs):
    running_loss = 0.0
    for images, label in trainloader:
        images, label = images.to(device), label.to(device)
        optimizer.zero_grad()
        output = model_tl(images)
        loss = crit(label, output)
        loss.backward()
        optimizer.step()
        running_loss += loss.item
    print(f" Ecpoch: {epoch} out of {epochs} ")
        

## ✅ Conclusion
- We used a **pretrained ResNet18** as a **feature extractor**.
- We replaced the top layer and trained it on a small dataset.
- You just performed **transfer learning** in PyTorch! 🎉